In [6]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [7]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Hyper-parameters
batch_size = 128
learning_rate = 1e-4
num_classes = 100
num_epochs = 25

In [9]:
# CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='data', 
                                              train=True, 
                                              transform=transforms.ToTensor(),  
                                              download=True)
test_dataset = torchvision.datasets.CIFAR100(root='data', 
                                             train=False, 
                                             transform=transforms.ToTensor())

Files already downloaded and verified


In [10]:
# CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='data', 
                                              train=True, 
                                              transform=transforms.ToTensor(),  
                                              download=True)
test_dataset = torchvision.datasets.CIFAR100(root='data', 
                                             train=False, 
                                             transform=transforms.ToTensor())

Files already downloaded and verified


In [15]:
# Data Loaders 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [29]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(8*8*32, num_classes)
        
    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [31]:
cnn = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [33]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/25], Step [100/391], Loss: 4.0264
Epoch [1/25], Step [200/391], Loss: 3.7821
Epoch [1/25], Step [300/391], Loss: 3.5991
Epoch [2/25], Step [100/391], Loss: 3.5124
Epoch [2/25], Step [200/391], Loss: 3.3348
Epoch [2/25], Step [300/391], Loss: 3.2977
Epoch [3/25], Step [100/391], Loss: 3.0110
Epoch [3/25], Step [200/391], Loss: 2.9691
Epoch [3/25], Step [300/391], Loss: 3.0231
Epoch [4/25], Step [100/391], Loss: 2.6936
Epoch [4/25], Step [200/391], Loss: 2.9009
Epoch [4/25], Step [300/391], Loss: 2.6784
Epoch [5/25], Step [100/391], Loss: 2.7538
Epoch [5/25], Step [200/391], Loss: 2.5861
Epoch [5/25], Step [300/391], Loss: 2.6214
Epoch [6/25], Step [100/391], Loss: 2.4945
Epoch [6/25], Step [200/391], Loss: 2.6787
Epoch [6/25], Step [300/391], Loss: 2.3611
Epoch [7/25], Step [100/391], Loss: 2.4416
Epoch [7/25], Step [200/391], Loss: 2.5427
Epoch [7/25], Step [300/391], Loss: 2.5548
Epoch [8/25], Step [100/391], Loss: 2.1737
Epoch [8/25], Step [200/391], Loss: 2.3827
Epoch [8/25

In [38]:
# Test the model
cnn.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 39.5 %


In [ ]:
# Save the model checkpoint
# torch.save(cnn.state_dict(), 'ckpts/cnn_cls.ckpt')